# Patching with Writes


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import json
import os


def detect_corners_harris(gray, block_size=2, ksize=3, k=0.04, threshold_ratio=0.01):
    gray_float = np.float32(gray)
    dst = cv2.cornerHarris(gray_float, blockSize=block_size, ksize=ksize, k=k)
    corner_mask = dst > threshold_ratio * dst.max()
    return corner_mask, dst


def count_corners_in_region(corner_mask, x, y, w, h):
    region = corner_mask[y:y+h, x:x+w]
    return np.sum(region)


def compute_corner_density(corner_mask, x, y, w, h):
    area = max(w * h, 1)
    count = count_corners_in_region(corner_mask, x, y, w, h)
    return count / area


def adaptive_subdivide(corner_mask, x, y, w, h,
                       density_threshold=0.005,
                       min_patch_width=32,
                       max_patch_width=256,
                       aspect_ratio=2.0):
    min_patch_height = int(min_patch_width * aspect_ratio)

    if w <= min_patch_width or h <= min_patch_height:
        return [(x, y, w, h)]

    density = compute_corner_density(corner_mask, x, y, w, h)

    min_w_check = max_patch_width // 2
    min_h_check = int(min_w_check * aspect_ratio)

    if density <= density_threshold or \
       (w <= min_w_check and h <= min_h_check and density <= density_threshold * 2):
        return [(x, y, w, h)]

    half_w = w // 2
    half_h = h // 2
    right_w = w - half_w
    bottom_h = h - half_h

    patches = []
    patches += adaptive_subdivide(corner_mask, x, y,
                                   half_w, half_h,
                                   density_threshold, min_patch_width,
                                   max_patch_width, aspect_ratio)
    patches += adaptive_subdivide(corner_mask, x + half_w, y,
                                   right_w, half_h,
                                   density_threshold, min_patch_width,
                                   max_patch_width, aspect_ratio)
    patches += adaptive_subdivide(corner_mask, x, y + half_h,
                                   half_w, bottom_h,
                                   density_threshold, min_patch_width,
                                   max_patch_width, aspect_ratio)
    patches += adaptive_subdivide(corner_mask, x + half_w, y + half_h,
                                   right_w, bottom_h,
                                   density_threshold, min_patch_width,
                                   max_patch_width, aspect_ratio)
    return patches


# ============================================================
#              EDGE-AWARE MERGING
# ============================================================

def compute_edge_map(gray):
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    edges = cv2.Canny(blurred, 50, 150)
    return edges


def get_shared_boundary(patch_a, patch_b):
    ax, ay, aw, ah = patch_a
    bx, by, bw, bh = patch_b
    tolerance = 2

    if abs((ay + ah) - by) <= tolerance:
        ol = max(ax, bx)
        or_ = min(ax + aw, bx + bw)
        if or_ - ol > tolerance:
            return ('horizontal', ol, or_, ay + ah)

    if abs((by + bh) - ay) <= tolerance:
        ol = max(ax, bx)
        or_ = min(ax + aw, bx + bw)
        if or_ - ol > tolerance:
            return ('horizontal', ol, or_, by + bh)

    if abs((ax + aw) - bx) <= tolerance:
        ot = max(ay, by)
        ob = min(ay + ah, by + bh)
        if ob - ot > tolerance:
            return ('vertical', ot, ob, ax + aw)

    if abs((bx + bw) - ax) <= tolerance:
        ot = max(ay, by)
        ob = min(ay + ah, by + bh)
        if ob - ot > tolerance:
            return ('vertical', ot, ob, bx + bw)

    return None


def compute_boundary_edge_density(edges, boundary_info, band_width=5):
    orientation, start, end, line_pos = boundary_info
    h, w = edges.shape

    if orientation == 'horizontal':
        rs = max(0, line_pos - band_width)
        re = min(h, line_pos + band_width)
        band = edges[rs:re, start:end]
    else:
        cs = max(0, line_pos - band_width)
        ce = min(w, line_pos + band_width)
        band = edges[start:end, cs:ce]

    if band.size == 0:
        return 0.0
    return np.sum(band > 0) / band.size


def merged_patch_bbox(patch_a, patch_b):
    ax, ay, aw, ah = patch_a
    bx, by, bw, bh = patch_b
    new_x = min(ax, bx)
    new_y = min(ay, by)
    new_x2 = max(ax + aw, bx + bw)
    new_y2 = max(ay + ah, by + bh)
    return (new_x, new_y, new_x2 - new_x, new_y2 - new_y)


def should_merge(patch_a, patch_b, edges, max_patch_width, max_patch_height,
                 edge_density_threshold=0.15, band_width=5):
    boundary = get_shared_boundary(patch_a, patch_b)
    if boundary is None:
        return False, None, 0.0

    merged = merged_patch_bbox(patch_a, patch_b)
    _, _, mw, mh = merged

    if mw > max_patch_width or mh > max_patch_height:
        return False, boundary, 0.0

    edge_density = compute_boundary_edge_density(edges, boundary, band_width)

    if edge_density > edge_density_threshold:
        return True, boundary, edge_density

    return False, boundary, edge_density


def edge_aware_merge(patches, edges, max_patch_width, max_patch_height,
                     edge_density_threshold=0.15,
                     band_width=5,
                     max_iterations=50):
    merged_patches = list(patches)
    total_merges = 0

    for iteration in range(max_iterations):
        best_merge = None
        best_density = 0.0

        for i in range(len(merged_patches)):
            for j in range(i + 1, len(merged_patches)):
                do_merge, boundary, density = should_merge(
                    merged_patches[i], merged_patches[j],
                    edges, max_patch_width, max_patch_height,
                    edge_density_threshold, band_width
                )
                if do_merge and density > best_density:
                    best_merge = (i, j)
                    best_density = density

        if best_merge is None:
            break

        i, j = best_merge
        new_patch = merged_patch_bbox(merged_patches[i], merged_patches[j])
        merged_patches.pop(j)
        merged_patches.pop(i)
        merged_patches.append(new_patch)
        total_merges += 1

    print(f"  Edge-aware merging: {total_merges} merges in {iteration+1} iterations")
    print(f"  Patches: {len(patches)} → {len(merged_patches)}")
    return merged_patches


# ============================================================
#         PATCH EXPORT & RECONSTRUCTION (NEW)
# ============================================================

def export_patches(image, patches, patch_info, output_dir='patches',
                   image_path=None):
    """
    Export each patch as an image file + metadata JSON for reconstruction.

    Directory structure:
      output_dir/
        metadata.json        ← coordinates & original image info
        patch_000.png        ← cropped patch image
        patch_001.png
        ...
    """
    os.makedirs(output_dir, exist_ok=True)

    height, width = image.shape[:2]
    channels = image.shape[2] if len(image.shape) == 3 else 1

    metadata = {
        'original_image': image_path if image_path else 'unknown',
        'original_width': width,
        'original_height': height,
        'original_channels': channels,
        'num_patches': len(patches),
        'patches': []
    }

    for i, (x, y, w, h) in enumerate(patches):
        # Crop the patch from the original image
        patch_img = image[y:y+h, x:x+w].copy()

        # Save patch image
        filename = f"patch_{i:03d}.png"
        filepath = os.path.join(output_dir, filename)
        cv2.imwrite(filepath, patch_img)

        # Find corresponding patch_info
        info = None
        for p in patch_info:
            if p['x'] == x and p['y'] == y and p['w'] == w and p['h'] == h:
                info = p
                break

        # Store metadata
        patch_meta = {
            'index': i,
            'filename': filename,
            'x': int(x),
            'y': int(y),
            'w': int(w),
            'h': int(h),
            'density': float(info['density']) if info else 0.0,
            'corner_count': int(info['corner_count']) if info else 0
        }
        metadata['patches'].append(patch_meta)

    # Save metadata
    meta_path = os.path.join(output_dir, 'metadata.json')
    with open(meta_path, 'w') as f:
        json.dump(metadata, f, indent=2)

    print(f"\n  Exported {len(patches)} patches to '{output_dir}/'")
    print(f"  Metadata saved to '{meta_path}'")
    print(f"  Total files: {len(patches)} images + 1 metadata JSON")

    # Print file sizes
    total_size = 0
    for i in range(len(patches)):
        fpath = os.path.join(output_dir, f"patch_{i:03d}.png")
        fsize = os.path.getsize(fpath)
        total_size += fsize

    meta_size = os.path.getsize(meta_path)
    total_size += meta_size
    print(f"  Total size: {total_size / 1024:.1f} KB "
          f"(patches: {(total_size - meta_size) / 1024:.1f} KB, "
          f"metadata: {meta_size / 1024:.1f} KB)")

    return metadata


def reconstruct_from_patches(patch_dir='patches', show_result=True):
    """
    Reconstruct the original image from exported patches + metadata.

    Reads metadata.json to know:
      - Original image dimensions
      - Exact (x, y, w, h) for each patch

    Then pastes each patch image at its exact location.
    """
    # Load metadata
    meta_path = os.path.join(patch_dir, 'metadata.json')
    if not os.path.exists(meta_path):
        raise FileNotFoundError(f"Metadata not found at {meta_path}")

    with open(meta_path, 'r') as f:
        metadata = json.load(f)

    orig_w = metadata['original_width']
    orig_h = metadata['original_height']
    orig_c = metadata['original_channels']
    num_patches = metadata['num_patches']

    print(f"Reconstructing image: {orig_w} x {orig_h} x {orig_c}")
    print(f"Using {num_patches} patches from '{patch_dir}/'")

    # Create empty canvas
    if orig_c == 1:
        canvas = np.zeros((orig_h, orig_w), dtype=np.uint8)
    else:
        canvas = np.zeros((orig_h, orig_w, orig_c), dtype=np.uint8)

    # Optional: track coverage to verify no gaps/overlaps
    coverage = np.zeros((orig_h, orig_w), dtype=np.int32)

    # Paste each patch
    for patch_meta in metadata['patches']:
        filename = patch_meta['filename']
        x = patch_meta['x']
        y = patch_meta['y']
        w = patch_meta['w']
        h = patch_meta['h']

        # Load patch image
        patch_path = os.path.join(patch_dir, filename)
        if not os.path.exists(patch_path):
            print(f"  WARNING: Missing patch file {patch_path}")
            continue

        patch_img = cv2.imread(patch_path, cv2.IMREAD_UNCHANGED)

        if patch_img is None:
            print(f"  WARNING: Could not read {patch_path}")
            continue

        # Verify dimensions match metadata
        if orig_c == 1 and len(patch_img.shape) == 3:
            patch_img = cv2.cvtColor(patch_img, cv2.COLOR_BGR2GRAY)

        patch_h, patch_w = patch_img.shape[:2]
        if patch_w != w or patch_h != h:
            print(f"  WARNING: Patch {filename} size mismatch: "
                  f"expected {w}x{h}, got {patch_w}x{patch_h}")
            # Crop/pad to expected size just in case
            patch_img = patch_img[:h, :w]

        # Paste onto canvas at exact coordinates
        canvas[y:y+h, x:x+w] = patch_img
        coverage[y:y+h, x:x+w] += 1

    # Check coverage
    uncovered = np.sum(coverage == 0)
    overlapped = np.sum(coverage > 1)
    total_pixels = orig_w * orig_h

    print(f"\n  Coverage analysis:")
    print(f"    Covered exactly once: {np.sum(coverage == 1)} / {total_pixels} "
          f"({100 * np.sum(coverage == 1) / total_pixels:.2f}%)")
    if uncovered > 0:
        print(f"    Uncovered pixels: {uncovered} "
              f"({100 * uncovered / total_pixels:.2f}%)")
    if overlapped > 0:
        print(f"    Overlapping pixels: {overlapped} "
              f"({100 * overlapped / total_pixels:.2f}%)")

    if show_result:
        print("\n--- Reconstructed Image ---")
        cv2_imshow(canvas)

    return canvas, coverage, metadata


def verify_reconstruction(original_image_path, patch_dir='patches'):
    """
    Verify that reconstruction is pixel-perfect by comparing
    with the original image.
    """
    original = cv2.imread(original_image_path)
    if original is None:
        raise ValueError(f"Cannot load original: {original_image_path}")

    reconstructed, coverage, metadata = reconstruct_from_patches(
        patch_dir, show_result=False
    )

    # Pixel-by-pixel comparison
    if original.shape != reconstructed.shape:
        print(f"\n  SHAPE MISMATCH: original {original.shape} vs "
              f"reconstructed {reconstructed.shape}")
        return False

    diff = cv2.absdiff(original, reconstructed)
    max_diff = np.max(diff)
    mean_diff = np.mean(diff)
    num_different = np.sum(diff > 0)
    total_values = diff.size

    print(f"\n  Verification:")
    print(f"    Max pixel difference: {max_diff}")
    print(f"    Mean pixel difference: {mean_diff:.6f}")
    print(f"    Different values: {num_different} / {total_values}")

    if max_diff == 0:
        print(f"    ✅ PERFECT reconstruction! Zero difference.")
        is_perfect = True
    else:
        print(f"    ⚠️  Some differences detected (likely PNG compression)")
        is_perfect = False

    # Show comparison
    print("\n--- Original vs Reconstructed ---")

    # Side by side (resize if too large)
    h, w = original.shape[:2]
    scale = min(1.0, 800 / w)
    if scale < 1.0:
        orig_small = cv2.resize(original, None, fx=scale, fy=scale)
        recon_small = cv2.resize(reconstructed, None, fx=scale, fy=scale)
    else:
        orig_small = original
        recon_small = reconstructed

    comparison = np.hstack([orig_small, recon_small])
    cv2.putText(comparison, "Original", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(comparison, "Reconstructed", (orig_small.shape[1] + 10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2_imshow(comparison)

    # Show difference map (amplified)
    if max_diff > 0:
        diff_amplified = (diff * (255.0 / max(max_diff, 1))).astype(np.uint8)
        print("\n--- Difference Map (amplified) ---")
        cv2_imshow(diff_amplified)

    return is_perfect


# ============================================================
#                    MAIN PIPELINE
# ============================================================

def generate_adaptive_patches(image_path,
                               density_threshold=0.005,
                               min_patch_width=32,
                               max_patch_width=256,
                               aspect_ratio=2.0,
                               harris_threshold_ratio=0.01,
                               edge_merge_threshold=0.15,
                               edge_band_width=5,
                               output_dir='patches'):
    """
    Full pipeline: detect → subdivide → merge → export → verify.
    """
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found at {image_path}")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape

    max_patch_height = int(max_patch_width * aspect_ratio)

    print(f"Image size: {width} x {height}")
    print(f"Patch size range: {min_patch_width}x{int(min_patch_width * aspect_ratio)} "
          f"to {max_patch_width}x{max_patch_height}")

    # ── Step 1: Harris Corners ──────────────────────────────
    print("\n[Step 1] Detecting Harris corners...")
    corner_mask, dst = detect_corners_harris(gray, threshold_ratio=harris_threshold_ratio)
    print(f"  Total corners detected: {np.sum(corner_mask)}")

    # ── Step 2: Adaptive Subdivision ────────────────────────
    print("\n[Step 2] Adaptive subdivision (vertical rectangles)...")
    patches = adaptive_subdivide(
        corner_mask,
        x=0, y=0, w=width, h=height,
        density_threshold=density_threshold,
        min_patch_width=min_patch_width,
        max_patch_width=max_patch_width,
        aspect_ratio=aspect_ratio
    )
    print(f"  Patches after subdivision: {len(patches)}")

    # ── Step 3: Edge-Aware Merging ──────────────────────────
    print("\n[Step 3] Edge-aware merging...")
    edges = compute_edge_map(gray)
    patches_before = len(patches)

    patches = edge_aware_merge(
        patches, edges,
        max_patch_width=max_patch_width,
        max_patch_height=max_patch_height,
        edge_density_threshold=edge_merge_threshold,
        band_width=edge_band_width
    )

    # ── Step 4: Build patch info ────────────────────────────
    patch_info = []
    for (x, y, w, h) in patches:
        density = compute_corner_density(corner_mask, x, y, w, h)
        count = count_corners_in_region(corner_mask, x, y, w, h)
        patch_info.append({
            'x': x, 'y': y, 'w': w, 'h': h,
            'density': density,
            'corner_count': count,
            'area': w * h,
            'aspect': h / max(w, 1)
        })
    patch_info.sort(key=lambda p: p['density'], reverse=True)

    # ── Step 5: Visualization ───────────────────────────────
    visualize_results(image, corner_mask, edges, patches, patch_info, patches_before)

    # ── Step 6: Export Patches ──────────────────────────────
    print("\n[Step 6] Exporting patches...")
    metadata = export_patches(image, patches, patch_info,
                              output_dir=output_dir,
                              image_path=image_path)

    # ── Step 7: Verify Reconstruction ───────────────────────
    print("\n[Step 7] Verifying reconstruction...")
    is_perfect = verify_reconstruction(image_path, patch_dir=output_dir)

    return patches, patch_info, corner_mask, metadata


def visualize_results(image, corner_mask, edges, patches, patch_info, patches_before):

    densities = [p['density'] for p in patch_info]
    max_density = max(densities) if max(densities) > 0 else 1

    def density_color(ratio):
        if ratio < 0.5:
            r, g, b = int(255 * ratio * 2), 255, 0
        else:
            r, g, b = 255, int(255 * (1 - (ratio - 0.5) * 2)), 0
        return (b, g, r)

    # Harris Corners
    corners_vis = image.copy()
    dst_dilated = cv2.dilate(corner_mask.astype(np.uint8) * 255, None)
    corners_vis[dst_dilated > 0] = [0, 0, 255]
    print("\n--- Harris Corners ---")
    cv2_imshow(corners_vis)

    # Patches (density colored)
    patches_vis = image.copy()
    for p in patch_info:
        x, y, w, h = p['x'], p['y'], p['w'], p['h']
        ratio = min(p['density'] / max_density, 1.0)
        cv2.rectangle(patches_vis, (x, y), (x+w, y+h), density_color(ratio), 2)
    print(f"\n--- Patches ({patches_before}→{len(patches)}) ---")
    cv2_imshow(patches_vis)

    # Heatmap
    alpha_layer = image.copy()
    for p in patch_info:
        x, y, w, h = p['x'], p['y'], p['w'], p['h']
        ratio = min(p['density'] / max_density, 1.0)
        cv2.rectangle(alpha_layer, (x, y), (x+w, y+h), density_color(ratio), -1)
    result = cv2.addWeighted(image, 0.6, alpha_layer, 0.4, 0)
    for p in patch_info:
        cv2.rectangle(result, (p['x'], p['y']),
                      (p['x']+p['w'], p['y']+p['h']), (255, 255, 255), 1)
    print("\n--- Heatmap ---")
    cv2_imshow(result)

    # Sizes
    size_vis = image.copy()
    areas = [p['area'] for p in patch_info]
    min_a, max_a = min(areas), max(areas)
    ar = max(max_a - min_a, 1)
    for p in patch_info:
        x, y, w, h = p['x'], p['y'], p['w'], p['h']
        ratio = (p['area'] - min_a) / ar
        cv2.rectangle(size_vis, (x, y), (x+w, y+h),
                      (int(255*ratio), 50, int(255*(1-ratio))), 2)
        if w > 40 and h > 30:
            cv2.putText(size_vis, f"{w}x{h}", (x+3, y+15),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1)
    print("\n--- Sizes ---")
    cv2_imshow(size_vis)

    print(f"\n--- Stats ---")
    print(f"Patches: {len(patches)}, Merged: {patches_before - len(patches)}")
    print(f"Non-square: {sum(1 for p in patch_info if p['w'] != p['h'])}")
    print(f"Avg aspect: {np.mean([p['aspect'] for p in patch_info]):.2f}")




In [ ]:
# ============================================================
#                       RUN IT
# ============================================================

# === STEP 1: Generate, export, and verify ===
patches, patch_info, corner_mask, metadata = generate_adaptive_patches(
    image_path='/content/img0.png',
    density_threshold=0.03,
    min_patch_width=32,
    max_patch_width=256,
    aspect_ratio=2.0,
    harris_threshold_ratio=0.01,
    edge_merge_threshold=0.15,
    edge_band_width=5,
    output_dir='/content/patches'
)

# # === STEP 2: Later, reconstruct from patches directory ===
# print("\n" + "="*60)
# print("RECONSTRUCTION FROM FILES")
# print("="*60)
# reconstructed, coverage, meta = reconstruct_from_patches(
#     patch_dir='patches',
#     show_result=True
# )

# # Save reconstructed image
# cv2.imwrite('reconstructed.png', reconstructed)
# print("Saved reconstructed.png")

# Setup Model

In [ ]:
# --- System deps ---
!apt-get update
!apt-get install -y swig

# --- Python deps ---
!pip install gdown


In [ ]:
!git clone https://github.com/tensorboy/pytorch_Realtime_Multi-Person_Pose_Estimation.git my_pose_repo
%cd my_pose_repo
!pip install -r requirements.txt


In [ ]:
# Create weight folder
!mkdir -p model/weight

# Download pretrained model
!gdown https://drive.google.com/uc?id=1_g4z0c1m5y7TY9FXki8uTf3JEtkTSSPF \
       -O model/weight/pose_model.pth

# Copy to repo root (expected by demo code)
!cp model/weight/pose_model.pth pose_model.pth
!ls -lh pose_model.pth


In [ ]:
%cd lib/pafprocess
!sh make.sh
%cd ../../


In [ ]:
import os
import json
import cv2
import subprocess
from tqdm import tqdm

PATCH_DIR = "/content/patches"
OUTPUT_DIR = "/content/patches_processed"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load metadata
with open(os.path.join(PATCH_DIR, "metadata.json"), "r") as f:
    metadata = json.load(f)

print(f"Processing {metadata['num_patches']} patches...")

for patch in tqdm(metadata["patches"]):
    patch_name = patch["filename"]
    patch_path = os.path.join(PATCH_DIR, patch_name)
    out_path = os.path.join(OUTPUT_DIR, patch_name)

    # Replace demo input image with patch path
    demo_file = "demo/picture_demo.py"

    with open(demo_file, "r") as f:
        lines = f.readlines()

    with open(demo_file, "w") as f:
        for line in lines:
            if "test_image =" in line:
                f.write(f'test_image = "{patch_path}"\n')
            else:
                f.write(line)

    # Run inference
    subprocess.run(
        ["python", "demo/picture_demo.py"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )

    # Save output as processed patch
    if os.path.exists("result.png"):
        result = cv2.imread("result.png")
        cv2.imwrite(out_path, result)
    else:
        print(f"⚠️ No result for {patch_name}, copying original")
        cv2.imwrite(out_path, cv2.imread(patch_path))

# Copy metadata unchanged
with open(os.path.join(OUTPUT_DIR, "metadata.json"), "w") as f:
    json.dump(metadata, f, indent=2)

print("✅ Patch inference complete")


In [ ]:
# Assuming reconstruct_from_patches is already defined (from your code)

reconstructed, coverage, meta = reconstruct_from_patches(
    patch_dir="/content/patches_processed",
    show_result=True
)

cv2.imwrite("reconstructed_pose.png", reconstructed)
print("Saved reconstructed_pose.png")


In [ ]:
# ===== CONFIG =====
IMAGE_PATH = "/content/img0.png"   # <-- CHANGE THIS
REPO_DIR = "/content/my_pose_repo"

# ==================
import os
import cv2
from IPython.display import Image, display
import subprocess

os.chdir(REPO_DIR)

# --- Update demo script to point to image ---
demo_file = "demo/picture_demo.py"

with open(demo_file, "r") as f:
    lines = f.readlines()

with open(demo_file, "w") as f:
    for line in lines:
        if "test_image =" in line:
            f.write(f'test_image = "{IMAGE_PATH}"\n')
        else:
            f.write(line)

# --- Run inference ---
subprocess.run(
    ["python", "demo/picture_demo.py"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# --- Display result ---
if os.path.exists("result.png"):
    print("✅ Pose estimation result:")
    display(Image("result.png"))
else:
    print("❌ result.png not found — inference failed")


# Patching by Depth

In [ ]:
!pip install -q transformers accelerate pillow opencv-python

In [ ]:
from transformers import pipeline

# Load depth model once
depth_pipe = pipeline(task="depth-estimation", model="depth-anything/Depth-Anything-V2-Small-hf")
print("✅ Depth model loaded")

In [ ]:
import cv2
import numpy as np
import json
import os
from PIL import Image
from google.colab.patches import cv2_imshow


# ============================================================
#              DEPTH COMPUTATION
# ============================================================

def compute_depth_map(image_bgr, depth_pipe):
    """
    Run depth estimation on a BGR image (OpenCV format).

    Returns:
      depth_norm: numpy array [0, 1], where 1 = farthest (deepest)
      depth_raw:  numpy array, raw predicted depth values
    """
    # Convert BGR (OpenCV) → RGB (PIL)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(image_rgb)

    # Run inference
    result = depth_pipe(pil_image)
    depth_tensor = result["predicted_depth"]  # torch tensor

    # Convert to numpy
    depth_raw = depth_tensor.squeeze().cpu().numpy()  # HxW

    # Resize to match original image dimensions (model may output different size)
    h, w = image_bgr.shape[:2]
    if depth_raw.shape[0] != h or depth_raw.shape[1] != w:
        depth_raw = cv2.resize(depth_raw, (w, h), interpolation=cv2.INTER_LINEAR)

    # Normalize to [0, 1]
    d_min, d_max = depth_raw.min(), depth_raw.max()
    if d_max - d_min > 1e-8:
        depth_norm = 1.0 - (depth_raw - d_min) / (d_max - d_min)
    else:
        depth_norm = np.zeros_like(depth_raw)

    return depth_norm, depth_raw


# ============================================================
#         DEPTH-BASED ADAPTIVE SUBDIVISION
# ============================================================

def compute_mean_depth(depth_map, x, y, w, h):
    region = depth_map[y:y+h, x:x+w]
    return np.mean(region)


def compute_depth_variance(depth_map, x, y, w, h):
    region = depth_map[y:y+h, x:x+w]
    return np.var(region)


def adaptive_subdivide_depth(depth_map, x, y, w, h,
                              depth_threshold=0.4,
                              min_patch_width=32,
                              max_patch_width=256,
                              aspect_ratio=2.0):
    """
    Recursively subdivide based on depth.
      - Deep regions (far away, value closer to 1) → smaller patches
      - Shallow regions (close, value closer to 0) → keep large
      - High depth variance → subdivide (mixed near/far)
    """
    min_patch_height = int(min_patch_width * aspect_ratio)

    if w <= min_patch_width or h <= min_patch_height:
        return [(x, y, w, h)]

    mean_depth = compute_mean_depth(depth_map, x, y, w, h)
    depth_var = compute_depth_variance(depth_map, x, y, w, h)

    mid_w = max_patch_width // 2
    mid_h = int(mid_w * aspect_ratio)

    # Don't subdivide if shallow and uniform
    if mean_depth <= depth_threshold and depth_var < 0.02:
        return [(x, y, w, h)]

    if (w <= mid_w and h <= mid_h and
        mean_depth <= depth_threshold * 1.3 and depth_var < 0.03):
        return [(x, y, w, h)]

    # Subdivide into 4 quadrants
    half_w = w // 2
    half_h = h // 2
    right_w = w - half_w
    bottom_h = h - half_h

    patches = []
    patches += adaptive_subdivide_depth(depth_map, x, y,
                                         half_w, half_h,
                                         depth_threshold, min_patch_width,
                                         max_patch_width, aspect_ratio)
    patches += adaptive_subdivide_depth(depth_map, x + half_w, y,
                                         right_w, half_h,
                                         depth_threshold, min_patch_width,
                                         max_patch_width, aspect_ratio)
    patches += adaptive_subdivide_depth(depth_map, x, y + half_h,
                                         half_w, bottom_h,
                                         depth_threshold, min_patch_width,
                                         max_patch_width, aspect_ratio)
    patches += adaptive_subdivide_depth(depth_map, x + half_w, y + half_h,
                                         right_w, bottom_h,
                                         depth_threshold, min_patch_width,
                                         max_patch_width, aspect_ratio)
    return patches


# ============================================================
#              EDGE-AWARE + DEPTH-AWARE MERGING
# ============================================================

def compute_edge_map(gray):
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    edges = cv2.Canny(blurred, 50, 150)
    return edges


def get_shared_boundary(patch_a, patch_b):
    ax, ay, aw, ah = patch_a
    bx, by, bw, bh = patch_b
    tolerance = 2

    if abs((ay + ah) - by) <= tolerance:
        ol = max(ax, bx)
        or_ = min(ax + aw, bx + bw)
        if or_ - ol > tolerance:
            return ('horizontal', ol, or_, ay + ah)

    if abs((by + bh) - ay) <= tolerance:
        ol = max(ax, bx)
        or_ = min(ax + aw, bx + bw)
        if or_ - ol > tolerance:
            return ('horizontal', ol, or_, by + bh)

    if abs((ax + aw) - bx) <= tolerance:
        ot = max(ay, by)
        ob = min(ay + ah, by + bh)
        if ob - ot > tolerance:
            return ('vertical', ot, ob, ax + aw)

    if abs((bx + bw) - ax) <= tolerance:
        ot = max(ay, by)
        ob = min(ay + ah, by + bh)
        if ob - ot > tolerance:
            return ('vertical', ot, ob, bx + bw)

    return None


def compute_boundary_edge_density(edges, boundary_info, band_width=5):
    orientation, start, end, line_pos = boundary_info
    h, w = edges.shape

    if orientation == 'horizontal':
        rs = max(0, line_pos - band_width)
        re = min(h, line_pos + band_width)
        band = edges[rs:re, start:end]
    else:
        cs = max(0, line_pos - band_width)
        ce = min(w, line_pos + band_width)
        band = edges[start:end, cs:ce]

    if band.size == 0:
        return 0.0
    return np.sum(band > 0) / band.size


def merged_patch_bbox(patch_a, patch_b):
    ax, ay, aw, ah = patch_a
    bx, by, bw, bh = patch_b
    new_x = min(ax, bx)
    new_y = min(ay, by)
    new_x2 = max(ax + aw, bx + bw)
    new_y2 = max(ay + ah, by + bh)
    return (new_x, new_y, new_x2 - new_x, new_y2 - new_y)


def should_merge_depth(patch_a, patch_b, depth_map, edges,
                        max_patch_width, max_patch_height,
                        depth_similarity_threshold=0.1,
                        edge_density_threshold=0.15,
                        band_width=5):
    boundary = get_shared_boundary(patch_a, patch_b)
    if boundary is None:
        return False, None, 0.0

    merged = merged_patch_bbox(patch_a, patch_b)
    _, _, mw, mh = merged
    if mw > max_patch_width or mh > max_patch_height:
        return False, boundary, 0.0

    depth_a = compute_mean_depth(depth_map, *patch_a)
    depth_b = compute_mean_depth(depth_map, *patch_b)
    depth_diff = abs(depth_a - depth_b)
    avg_depth = (depth_a + depth_b) / 2.0

    # Merge if both shallow and similar depth
    if depth_diff < depth_similarity_threshold and avg_depth < 0.5:
        return True, boundary, 1.0 - depth_diff

    # Also allow merge if no strong edge and depth is close enough
    edge_density = compute_boundary_edge_density(edges, boundary, band_width)
    if edge_density < edge_density_threshold and depth_diff < depth_similarity_threshold * 2:
        return True, boundary, 1.0 - depth_diff

    return False, boundary, 0.0


def depth_aware_merge(patches, depth_map, edges,
                       max_patch_width, max_patch_height,
                       depth_similarity_threshold=0.1,
                       edge_density_threshold=0.15,
                       band_width=5,
                       max_iterations=50):
    merged_patches = list(patches)
    total_merges = 0

    for iteration in range(max_iterations):
        best_merge = None
        best_score = 0.0

        for i in range(len(merged_patches)):
            for j in range(i + 1, len(merged_patches)):
                do_merge, boundary, score = should_merge_depth(
                    merged_patches[i], merged_patches[j],
                    depth_map, edges,
                    max_patch_width, max_patch_height,
                    depth_similarity_threshold,
                    edge_density_threshold, band_width
                )
                if do_merge and score > best_score:
                    best_merge = (i, j)
                    best_score = score

        if best_merge is None:
            break

        i, j = best_merge
        new_patch = merged_patch_bbox(merged_patches[i], merged_patches[j])
        merged_patches.pop(j)
        merged_patches.pop(i)
        merged_patches.append(new_patch)
        total_merges += 1

    print(f"  Depth-aware merging: {total_merges} merges in {iteration+1} iterations")
    print(f"  Patches: {len(patches)} → {len(merged_patches)}")
    return merged_patches


# ============================================================
#         EXPORT & RECONSTRUCTION
# ============================================================

def export_patches(image, patches, patch_info, output_dir='patches',
                   image_path=None):
    os.makedirs(output_dir, exist_ok=True)

    height, width = image.shape[:2]
    channels = image.shape[2] if len(image.shape) == 3 else 1

    metadata = {
        'original_image': image_path if image_path else 'unknown',
        'original_width': width,
        'original_height': height,
        'original_channels': channels,
        'num_patches': len(patches),
        'patches': []
    }

    for i, (x, y, w, h) in enumerate(patches):
        patch_img = image[y:y+h, x:x+w].copy()
        filename = f"patch_{i:03d}.png"
        filepath = os.path.join(output_dir, filename)
        cv2.imwrite(filepath, patch_img)

        info = None
        for p in patch_info:
            if p['x'] == x and p['y'] == y and p['w'] == w and p['h'] == h:
                info = p
                break

        patch_meta = {
            'index': i,
            'filename': filename,
            'x': int(x),
            'y': int(y),
            'w': int(w),
            'h': int(h),
            'mean_depth': float(info['mean_depth']) if info else 0.0,
            'depth_variance': float(info['depth_variance']) if info else 0.0
        }
        metadata['patches'].append(patch_meta)

    meta_path = os.path.join(output_dir, 'metadata.json')
    with open(meta_path, 'w') as f:
        json.dump(metadata, f, indent=2)

    print(f"\n  Exported {len(patches)} patches to '{output_dir}/'")

    total_size = sum(
        os.path.getsize(os.path.join(output_dir, f"patch_{i:03d}.png"))
        for i in range(len(patches))
    )
    meta_size = os.path.getsize(meta_path)
    total_size += meta_size
    print(f"  Total size: {total_size / 1024:.1f} KB")

    return metadata


def reconstruct_from_patches(patch_dir='patches', show_result=True):
    meta_path = os.path.join(patch_dir, 'metadata.json')
    with open(meta_path, 'r') as f:
        metadata = json.load(f)

    orig_w = metadata['original_width']
    orig_h = metadata['original_height']
    orig_c = metadata['original_channels']

    print(f"Reconstructing: {orig_w}x{orig_h}x{orig_c} from {metadata['num_patches']} patches")

    if orig_c == 1:
        canvas = np.zeros((orig_h, orig_w), dtype=np.uint8)
    else:
        canvas = np.zeros((orig_h, orig_w, orig_c), dtype=np.uint8)

    coverage = np.zeros((orig_h, orig_w), dtype=np.int32)

    for patch_meta in metadata['patches']:
        x, y, w, h = patch_meta['x'], patch_meta['y'], patch_meta['w'], patch_meta['h']
        patch_path = os.path.join(patch_dir, patch_meta['filename'])

        patch_img = cv2.imread(patch_path, cv2.IMREAD_UNCHANGED)
        if patch_img is None:
            print(f"  WARNING: Could not read {patch_path}")
            continue

        if orig_c == 1 and len(patch_img.shape) == 3:
            patch_img = cv2.cvtColor(patch_img, cv2.COLOR_BGR2GRAY)

        patch_img = patch_img[:h, :w]
        canvas[y:y+h, x:x+w] = patch_img
        coverage[y:y+h, x:x+w] += 1

    uncovered = np.sum(coverage == 0)
    total_pixels = orig_w * orig_h
    print(f"  Coverage: {np.sum(coverage == 1)}/{total_pixels} "
          f"({100 * np.sum(coverage == 1) / total_pixels:.2f}%)")
    if uncovered > 0:
        print(f"  ⚠️ Uncovered: {uncovered} pixels")

    if show_result:
        cv2_imshow(canvas)

    return canvas, coverage, metadata


def verify_reconstruction(original_image_path, patch_dir='patches'):
    original = cv2.imread(original_image_path)
    reconstructed, _, _ = reconstruct_from_patches(patch_dir, show_result=False)

    if original.shape != reconstructed.shape:
        print(f"  SHAPE MISMATCH: {original.shape} vs {reconstructed.shape}")
        return False

    diff = cv2.absdiff(original, reconstructed)
    max_diff = np.max(diff)

    if max_diff == 0:
        print("  ✅ PERFECT reconstruction!")
    else:
        print(f"  ⚠️ Max diff: {max_diff}, Mean: {np.mean(diff):.6f}")

    h, w = original.shape[:2]
    scale = min(1.0, 800 / w)
    orig_s = cv2.resize(original, None, fx=scale, fy=scale) if scale < 1 else original
    recon_s = cv2.resize(reconstructed, None, fx=scale, fy=scale) if scale < 1 else reconstructed
    comparison = np.hstack([orig_s, recon_s])
    cv2.putText(comparison, "Original", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(comparison, "Reconstructed", (orig_s.shape[1] + 10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2_imshow(comparison)
    return max_diff == 0


# ============================================================
#         VISUALIZATION
# ============================================================

def visualize_depth_patches(image, depth_norm, patches, patch_info, patches_before):
    def depth_color(ratio):
        b = int(255 * (1 - ratio))
        r = int(255 * ratio)
        return (b, 0, r)

    # Depth map
    depth_vis = (depth_norm * 255).astype(np.uint8)
    depth_colored = cv2.applyColorMap(depth_vis, cv2.COLORMAP_INFERNO)
    print("\n--- Depth Map ---")
    cv2_imshow(depth_colored)

    # Patches colored by depth
    patches_vis = image.copy()
    for p in patch_info:
        x, y, w, h = p['x'], p['y'], p['w'], p['h']
        cv2.rectangle(patches_vis, (x, y), (x+w, y+h),
                      depth_color(min(p['mean_depth'], 1.0)), 2)
    print(f"\n--- Patches ({patches_before}→{len(patches)}) by depth ---")
    cv2_imshow(patches_vis)

    # Overlay
    alpha_layer = image.copy()
    for p in patch_info:
        x, y, w, h = p['x'], p['y'], p['w'], p['h']
        cv2.rectangle(alpha_layer, (x, y), (x+w, y+h),
                      depth_color(min(p['mean_depth'], 1.0)), -1)
    result = cv2.addWeighted(image, 0.6, alpha_layer, 0.4, 0)
    for p in patch_info:
        cv2.rectangle(result, (p['x'], p['y']),
                      (p['x']+p['w'], p['y']+p['h']), (255, 255, 255), 1)
    print("\n--- Depth Overlay ---")
    cv2_imshow(result)

    # Sizes
    size_vis = image.copy()
    areas = [p['area'] for p in patch_info]
    min_a, max_a = min(areas), max(areas)
    ar = max(max_a - min_a, 1)
    for p in patch_info:
        x, y, w, h = p['x'], p['y'], p['w'], p['h']
        ratio = (p['area'] - min_a) / ar
        cv2.rectangle(size_vis, (x, y), (x+w, y+h),
                      (int(255*ratio), 50, int(255*(1-ratio))), 2)
        if w > 40 and h > 30:
            cv2.putText(size_vis, f"{w}x{h}", (x+3, y+15),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1)
    print("\n--- Patch Sizes ---")
    cv2_imshow(size_vis)

    depths = [p['mean_depth'] for p in patch_info]
    print(f"\n--- Stats ---")
    print(f"Patches: {len(patches)} | Merged: {patches_before - len(patches)}")
    print(f"Depth range: [{min(depths):.3f}, {max(depths):.3f}]")
    print(f"Avg aspect: {np.mean([p['aspect'] for p in patch_info]):.2f}")


# ============================================================
#                    MAIN PIPELINE
# ============================================================

def generate_depth_adaptive_patches(image_path,
                                     depth_pipe,
                                     depth_threshold=0.4,
                                     min_patch_width=32,
                                     max_patch_width=256,
                                     aspect_ratio=2.0,
                                     depth_similarity_threshold=0.1,
                                     edge_density_threshold=0.15,
                                     edge_band_width=5,
                                     output_dir='patches_depth'):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found: {image_path}")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape
    max_patch_height = int(max_patch_width * aspect_ratio)

    print(f"Image: {width}x{height}")
    print(f"Patch range: {min_patch_width}x{int(min_patch_width*aspect_ratio)} "
          f"→ {max_patch_width}x{max_patch_height}")
    print(f"Depth threshold: {depth_threshold}")

    # Step 1: Depth map
    print("\n[Step 1] Computing depth...")
    depth_norm, depth_raw = compute_depth_map(image, depth_pipe)
    print(f"  Raw range: [{depth_raw.min():.2f}, {depth_raw.max():.2f}]")
    print(f"  Mean (norm): {depth_norm.mean():.3f}")

    # Step 2: Subdivide
    print("\n[Step 2] Depth-adaptive subdivision...")
    patches = adaptive_subdivide_depth(
        depth_norm, 0, 0, width, height,
        depth_threshold, min_patch_width, max_patch_width, aspect_ratio
    )
    print(f"  Patches: {len(patches)}")

    # Step 3: Merge
    print("\n[Step 3] Depth-aware merging...")
    edges = compute_edge_map(gray)
    patches_before = len(patches)
    patches = depth_aware_merge(
        patches, depth_norm, edges,
        max_patch_width, max_patch_height,
        depth_similarity_threshold, edge_density_threshold, edge_band_width
    )

    # Step 4: Patch info
    patch_info = []
    for (x, y, w, h) in patches:
        patch_info.append({
            'x': x, 'y': y, 'w': w, 'h': h,
            'mean_depth': compute_mean_depth(depth_norm, x, y, w, h),
            'depth_variance': compute_depth_variance(depth_norm, x, y, w, h),
            'area': w * h,
            'aspect': h / max(w, 1)
        })
    patch_info.sort(key=lambda p: p['mean_depth'], reverse=True)

    # Step 5: Visualize
    visualize_depth_patches(image, depth_norm, patches, patch_info, patches_before)

    # Step 6: Export
    print("\n[Step 6] Exporting...")
    metadata = export_patches(image, patches, patch_info,
                              output_dir=output_dir, image_path=image_path)

    # Step 7: Verify
    print("\n[Step 7] Verifying...")
    verify_reconstruction(image_path, patch_dir=output_dir)

    return patches, patch_info, depth_norm, metadata


print("✅ Depth-based patching pipeline ready")

In [ ]:
IMAGE_PATH = '/content/img0.png'  # ← change this

patches, patch_info, depth_map, metadata = generate_depth_adaptive_patches(
    image_path=IMAGE_PATH,
    depth_pipe=depth_pipe,           # from Cell 1
    depth_threshold=0.7,             # lower = more subdivision
    min_patch_width=32,
    max_patch_width=256,
    aspect_ratio=2.0,
    depth_similarity_threshold=0.1,
    edge_density_threshold=0.15,
    output_dir='/content/patches_depth'
)

In [ ]:
import os
import json
import cv2
import subprocess
from tqdm import tqdm

PATCH_DIR = "/content/patches_depth"
OUTPUT_DIR = "/content/patches_depth_processed"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load metadata
with open(os.path.join(PATCH_DIR, "metadata.json"), "r") as f:
    metadata = json.load(f)

print(f"Processing {metadata['num_patches']} patches...")

for patch in tqdm(metadata["patches"]):
    patch_name = patch["filename"]
    patch_path = os.path.join(PATCH_DIR, patch_name)
    out_path = os.path.join(OUTPUT_DIR, patch_name)

    # Replace demo input image with patch path
    demo_file = "demo/picture_demo.py"

    with open(demo_file, "r") as f:
        lines = f.readlines()

    with open(demo_file, "w") as f:
        for line in lines:
            if "test_image =" in line:
                f.write(f'test_image = "{patch_path}"\n')
            else:
                f.write(line)

    # Run inference
    subprocess.run(
        ["python", "demo/picture_demo.py"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )

    # Save output as processed patch
    if os.path.exists("result.png"):
        result = cv2.imread("result.png")
        cv2.imwrite(out_path, result)
    else:
        print(f"⚠️ No result for {patch_name}, copying original")
        cv2.imwrite(out_path, cv2.imread(patch_path))

# Copy metadata unchanged
with open(os.path.join(OUTPUT_DIR, "metadata.json"), "w") as f:
    json.dump(metadata, f, indent=2)

print("✅ Patch inference complete")


In [ ]:
# Assuming reconstruct_from_patches is already defined (from your code)

reconstructed, coverage, meta = reconstruct_from_patches(
    patch_dir="/content/patches_depth_processed",
    show_result=True
)

cv2.imwrite("reconstructed_pose.png", reconstructed)
print("Saved reconstructed_pose.png")


# Hijab

In [ ]:
# ===== CONFIG =====
IMAGE_PATH = "/content/img3.jpg"   # <-- CHANGE THIS
REPO_DIR = "/content/my_pose_repo"

# ==================
import os
import cv2
from IPython.display import Image, display
import subprocess

os.chdir(REPO_DIR)

# --- Update demo script to point to image ---
demo_file = "demo/picture_demo.py"

with open(demo_file, "r") as f:
    lines = f.readlines()

with open(demo_file, "w") as f:
    for line in lines:
        if "test_image =" in line:
            f.write(f'test_image = "{IMAGE_PATH}"\n')
        else:
            f.write(line)

# --- Run inference ---
subprocess.run(
    ["python", "demo/picture_demo.py"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# --- Display result ---
if os.path.exists("result.png"):
    print("✅ Pose estimation result:")
    display(Image("result.png"))
else:
    print("❌ result.png not found — inference failed")


In [ ]:
import cv2

# input and output paths
input_path = "/content/img3.jpg"
output_path = "output_negative.jpg"

# read image
image = cv2.imread(input_path)

# check if image loaded
if image is None:
    raise ValueError("Could not load image. Check the input path.")

# create negative (invert colors)
negative = 255 - image

# save result
cv2.imwrite(output_path, negative)

print("Negative image saved to:", output_path)

In [ ]:
# ===== CONFIG =====
IMAGE_PATH = "output_negative.jpg"   # <-- CHANGE THIS
REPO_DIR = "/content/my_pose_repo"

# ==================
import os
import cv2
from IPython.display import Image, display
import subprocess

os.chdir(REPO_DIR)

# --- Update demo script to point to image ---
demo_file = "demo/picture_demo.py"

with open(demo_file, "r") as f:
    lines = f.readlines()

with open(demo_file, "w") as f:
    for line in lines:
        if "test_image =" in line:
            f.write(f'test_image = "{IMAGE_PATH}"\n')
        else:
            f.write(line)

# --- Run inference ---
subprocess.run(
    ["python", "demo/picture_demo.py"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# --- Display result ---
if os.path.exists("result.png"):
    print("✅ Pose estimation result:")
    display(Image("result.png"))
else:
    print("❌ result.png not found — inference failed")


In [ ]:
# ===== CONFIG =====
IMAGE_PATH = "/content/img4.jpg"   # <-- CHANGE THIS
REPO_DIR = "/content/my_pose_repo"

# ==================
import os
import cv2
from IPython.display import Image, display
import subprocess

os.chdir(REPO_DIR)

# --- Update demo script to point to image ---
demo_file = "demo/picture_demo.py"

with open(demo_file, "r") as f:
    lines = f.readlines()

with open(demo_file, "w") as f:
    for line in lines:
        if "test_image =" in line:
            f.write(f'test_image = "{IMAGE_PATH}"\n')
        else:
            f.write(line)

# --- Run inference ---
subprocess.run(
    ["python", "demo/picture_demo.py"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# --- Display result ---
if os.path.exists("result.png"):
    print("✅ Pose estimation result:")
    display(Image("result.png"))
else:
    print("❌ result.png not found — inference failed")


In [ ]:
# ===== CONFIG =====
IMAGE_PATH = "/content/img5.jpg"   # <-- CHANGE THIS
REPO_DIR = "/content/my_pose_repo"

# ==================
import os
import cv2
from IPython.display import Image, display
import subprocess

os.chdir(REPO_DIR)

# --- Update demo script to point to image ---
demo_file = "demo/picture_demo.py"

with open(demo_file, "r") as f:
    lines = f.readlines()

with open(demo_file, "w") as f:
    for line in lines:
        if "test_image =" in line:
            f.write(f'test_image = "{IMAGE_PATH}"\n')
        else:
            f.write(line)

# --- Run inference ---
subprocess.run(
    ["python", "demo/picture_demo.py"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# --- Display result ---
if os.path.exists("result.png"):
    print("✅ Pose estimation result:")
    display(Image("result.png"))
else:
    print("❌ result.png not found — inference failed")


In [ ]:
# ===== CONFIG =====
IMAGE_PATH = "/content/img6.jpg"   # <-- CHANGE THIS
REPO_DIR = "/content/my_pose_repo"

# ==================
import os
import cv2
from IPython.display import Image, display
import subprocess

os.chdir(REPO_DIR)

# --- Update demo script to point to image ---
demo_file = "demo/picture_demo.py"

with open(demo_file, "r") as f:
    lines = f.readlines()

with open(demo_file, "w") as f:
    for line in lines:
        if "test_image =" in line:
            f.write(f'test_image = "{IMAGE_PATH}"\n')
        else:
            f.write(line)

# --- Run inference ---
subprocess.run(
    ["python", "demo/picture_demo.py"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# --- Display result ---
if os.path.exists("result.png"):
    print("✅ Pose estimation result:")
    display(Image("result.png"))
else:
    print("❌ result.png not found — inference failed")
